## EDA. Preprocessing
### Version 2 (Финальная)

In [5]:
import polars as pl
import numpy as np
import matplotlib.pyplot as plt

pl.Config.set_fmt_str_lengths(100)

In [72]:
data = pl.read_csv("raw/Data_Sources_csv.csv", separator=";")
data.head(5)

url,date,text,tag,source
str,str,str,str,str
"""https://vk.com…","""2023-04-15 19:…","""Всем добрый де…","""БАР""","""ВК"""
"""https://vk.com…","""2023-04-15 19:…","""это на всякий …","""БАР""","""ВК"""
"""https://vk.com…","""2023-04-15 19:…","""Попробуйте усп…","""БАР""","""ВК"""
"""https://vk.com…","""2023-04-15 19:…","""На все вопросы…","""БАР""","""ВК"""
"""https://vk.com…","""2023-04-15 19:…","""Всё к лучшему.…","""БАР""","""ВК"""


Т.к. разрешено оставить не менее 6 классов, уберем "тревожное р-во/невроз", "тревожное р-во/депрессия", "паранойя"

In [73]:
data = data.drop_nulls()
data = data.filter(~pl.col("tag").is_in(["тревожное р-во/невроз", "тревожное р-во/депрессия", "паранойя"]))
data.shape

(62770, 5)

In [74]:
# pl.Config.set_fmt_str_lengths(100)
# data.select(pl.col(["url", "text", "tag"]).shuffle()).head(500)
# data.filter(pl.col("url") == "https://vk.com/id3452919")
# data.filtergroupby("url").count()

### Заводим аггрегационные признаки для дальнейшей фильтрации:
- Количество слов в документа
- Количество сообщений в рамках одного url

In [75]:
data = data.with_columns((pl.col('text').str.split(' ').list.len()).alias('word_count'))
topic_sizes = data.group_by("url").len().rename({"len": "topic_size"})
data = data.join(
    topic_sizes,
    on=['url'],
    how='left',
)

Фильтруем особым образом, не трогая классы ["тревожное р-во", "депрессия"], т.к. их меньше всего представлено. Убираем строки, где сообщения короткие(в них менее вероятно присутствует нужная нам семантика), а так же убираем топики, которые представлены малым числом сообщений. Второе нам пригодится в будущем. 

In [76]:
# data = data.filter(((pl.col("topic_size") == 1) & (pl.col("word_count") < 20)))
# data = data.filter((pl.col("topic_size") > 15) & (~pl.col("tag").is_in(["БАР", "шизофрения"])))
filtered_data = data.with_columns(
    pl.when(pl.col("tag").is_in(["тревожное р-во", "депрессия"]))
    .then(pl.col("word_count") > 20)
    .otherwise(True)
    .alias("filter_condition1")
).filter(pl.col("filter_condition1"))
filtered_data.drop("filter_condition1")

filtered_data = filtered_data.with_columns(
    pl.when(pl.col("tag").is_in(["тревожное р-во", "депрессия"]))
    .then(pl.col("topic_size") > 20)
    .otherwise(True)
    .alias("filter_condition2")
).filter(pl.col("filter_condition2"))
filtered_data.drop("filter_condition2")


url,date,text,tag,source,word_count,topic_size,filter_condition1
str,str,str,str,str,u32,u32,bool
"""https://vk.com…","""2023-04-15 19:…","""Всем добрый де…","""БАР""","""ВК""",169,1,true
"""https://vk.com…","""2023-04-15 19:…","""это на всякий …","""БАР""","""ВК""",4,32,true
"""https://vk.com…","""2023-04-15 19:…","""Попробуйте усп…","""БАР""","""ВК""",28,126,true
"""https://vk.com…","""2023-04-15 19:…","""На все вопросы…","""БАР""","""ВК""",6,14,true
"""https://vk.com…","""2023-04-15 19:…","""Всё к лучшему.…","""БАР""","""ВК""",24,2,true
"""https://vk.com…","""2023-04-15 20:…","""вы серьезно? А…","""БАР""","""ВК""",31,1,true
"""https://vk.com…","""2023-04-16 2:2…","""я не за болезн…","""БАР""","""ВК""",66,2,true
"""https://vk.com…","""2023-04-15 19:…","""Лучше сменить …","""БАР""","""ВК""",12,1,true
"""https://vk.com…","""2023-04-15 19:…","""Препараты, АД,…","""БАР""","""ВК""",30,1,true


In [77]:
print(filtered_data.shape)
filtered_data["tag"].value_counts()

(35704, 9)


tag,count
str,u32
"""ПРЛ""",5219
"""ОКР""",5158
"""тревожное р-во…",5094
"""шизофрения""",2055
"""депрессия""",15384
"""БАР""",2794


Теперь обрежем датасет до ~20 тыс. строк. Чистим равномерно по каждому классу, в приоритете оставляя тексты с наибольшей длиной

In [84]:
target_total_count = 24000

# количество уникальных тегов
unique_tags = filtered_data['tag'].n_unique()

# целевое количество строк для каждого класса
target_count_per_class = target_total_count // unique_tags

# функция для выборки подмножества данных для каждого класса по наибольшему количеству слов
def select_top_by_word_count(df: pl.DataFrame, target_count):
    return df.sort('word_count', descending=True).head(target_count)

balanced_dfs = []

for tag in filtered_data['tag'].unique():
    class_df = filtered_data.filter(pl.col('tag') == tag)
    sampled_class_df = select_top_by_word_count(class_df, target_count_per_class)
    balanced_dfs.append(sampled_class_df)

# объединяем все полученной в один датафрейм
balanced_df = pl.concat(balanced_dfs)

# Если суммарное количество строк больше нужного, случайно уменьшаем до нужного размера
if balanced_df.shape[0] > target_total_count:
    balanced_df = balanced_df.sample(n=target_total_count)

balanced_df = balanced_df.sample(fraction=1, shuffle=True)

Финальное распределение по классам выглядит следующим образом:

In [85]:
balanced_df["tag"].value_counts()

tag,count
str,u32
"""ОКР""",4000
"""депрессия""",4000
"""БАР""",2794
"""шизофрения""",2055
"""тревожное р-во…",4000
"""ПРЛ""",4000


Уложились в 20 т. строк

In [86]:
balanced_df.shape

(20849, 9)

### Разбиение на train/test

Разбиение проводим равномерно по каждому классу (stratify)

Сохраняем обработанный датасет и идем дальше.

In [87]:
tag_counts = balanced_df.groupby('tag').agg(pl.col('tag').count().alias('tag_count'))
balanced_df = balanced_df.join(tag_counts, on='tag', how='left')
balanced_df.head(3)

from sklearn.model_selection import train_test_split

X = balanced_df.select(["url", "date", "text", "source", "word_count", "topic_size", "tag_count"])
y = balanced_df.select("tag")

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=balanced_df["tag_count"].to_numpy())

X_train.write_parquet("preprocessed/v2/X_train.parquet")
y_train.write_parquet("preprocessed/v2/y_train.parquet")
X_test.write_parquet("preprocessed/v2/X_test.parquet")
y_test.write_parquet("preprocessed/v2/y_test.parquet")

/tmp/ipykernel_32357/3178772607.py:1: DeprecationWarning: `groupby` is deprecated. It has been renamed to `group_by`.
  tag_counts = balanced_df.groupby('tag').agg(pl.col('tag').count().alias('tag_count'))
